In [1]:
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cv2
import os
import pydicom
import numpy as np
from pathlib import Path
from sklearn.metrics import precision_score, roc_curve as roc, auc

**DEFINE PATHS AND LOAD PATIENT INFO CSVS**

In [2]:
patient_info_t1 = pd.read_pickle(str(Path('Methylation/patient_info_t1.pickle').absolute()))

In [3]:
patient_info_t1.head()

,Patient ID,modality_path,GivenModalityForThisStudy,count,frames_with_tumour
2,TCGA-02-0003,/external_drive/MRI_methylation/Methylation/da...,t1,25,"[000000.dcm, 000001.dcm, 000002.dcm, 000003.dc..."
6,TCGA-02-0006,/external_drive/MRI_methylation/Methylation/da...,t1,23,"[000000.dcm, 000001.dcm, 000002.dcm, 000003.dc..."
10,TCGA-02-0009,/external_drive/MRI_methylation/Methylation/da...,t1,23,"[000000.dcm, 000001.dcm, 000002.dcm, 000003.dc..."
14,TCGA-02-0011,/external_drive/MRI_methylation/Methylation/da...,t1,23,"[000000.dcm, 000001.dcm, 000002.dcm, 000003.dc..."
18,TCGA-02-0033,/external_drive/MRI_methylation/Methylation/da...,t1,23,"[000000.dcm, 000001.dcm, 000002.dcm, 000003.dc..."


**PREPROCESS INPUT DATA**

In [4]:
for index, row in patient_info_t1.iterrows():
    for brain_slice in os.listdir(row['modality_path']):
        if str(brain_slice) in row['frames_with_tumour']:
            try:
                #Load Raw Dicoms
                dcm = pydicom.dcmread(os.path.join(row['modality_path'],brain_slice))
                img = dcm.pixel_array
                original_img = img.copy()
                #If the meta data has a slope and intercept, use it to rescale the pixel values to Hounsfield Units
                try:
                    img = img*dcm.RescaleSlope+dcm.RescaleIntercept
                except:
                    pass
                 #If the meta data has a window parameter use it to define the window in which relevant pixels are found
                try:
                    min_value, max_value = dcm.WindowCenter - dcm.WindowWidth/2, dcm.WindowCenter + dcm.WindowWidth/2
                    img[img < min_value] = min_value
                    img[img > max_value] = max_value
                except:
                    pass
                #Filter out noisy images as defined by low mean and standard deviation of pixel vaues
                if (np.mean(img)<20) & (np.std(img)<20):
                    noisy_images.append(os.path.join(row['modality_path'],brain_slice))
                else:
#                     img = (img - np.mean(img))/np.std(img)
                    plt.figure(figsize=(10, 6.67))
                    plt.subplot(1, 2, 1)
                    plt.imshow(original_img, cmap="bone", origin="lower")
                    plt.title("Original Image")
                    plt.axis("off")
                    plt.subplot(1, 2, 2)
                    plt.imshow(img, cmap="bone", origin="lower")
                    plt.title("Preprocessed Image")
                    plt.axis("off")
                    os.makedirs(row['modality_path'].replace('TCGA-GBM','TCGA-GBM-pre-post_process'),exist_ok=True)
                    plt.savefig(os.path.join(row['modality_path'].replace('TCGA-GBM','TCGA-GBM-pre-post_process'),
                                         brain_slice.replace('dcm','jpg')), bbox_inches='tight')                  
            except Exception as e:
                print('****')
                print(e)
                print(os.path.join(row['modality_path'],brain_slice))
                print('****')

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pydicom/filereader.py:313: UserWarning: Expected explicit VR, but found implicit VR - using implicit VR for reading
  warnings.warn(message, UserWarning)
